# YouTube Sports Predictions

- **Question:** Which sport analyst is correct with the team they predict to win?
- **Tasks:**
    
    1. Find any pre-game YouTube clip where the sport analysts make predictions of which team they think will win.
        - Ex. See [Immediate 2025 NCAA tournament Final Four and championship picks](https://youtu.be/-rjnvL9LL3U?si=QMFJYAQ8Q85lTNCD)
    2. Find a post-game YouTube clip where the winning team (from 1) is announced.
        - Ex. See [Florida vs. Houston - 2025 NCAA men’s national championship | FULL REPLAY](https://www.youtube.com/watch?v=UHJ90jJmCYw)
    3. Use the [`youtube-transcript-api`](https://pypi.org/project/youtube-transcript-api/) to retrieve the transcript of both videos (from 1 and 2).
    4. Extract the transcripts
    5. Clean the data
    6. Create prompt to identify predictions.
    7. Pass each row from video 1 + prompt into the LLMs.
    8. Majority Vote (MV) with LLMs
    9. Filter by MV label = 1, hence prediction
    10. For each prediction, compare with each observation sentence from video 2. 
    11. Prompt LLMs to identify if observation from video 2 is enough to state if each prediction came true.
- **Misc:**
    
    1. Watch the video first to ensure it contains predictions (the team that is predicted to win) and observations (the team that wins)
    2. Shorten the transcript to only have predictions/observations

In [1]:
import os
import sys

import pandas as pd

from tqdm import tqdm
from youtube_transcript_api import YouTubeTranscriptApi

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))
# from metrics import Metrics
# from feature_extraction import SpacyFeatureExtraction
from text_generation_models import TextGenerationModelFactory, llm_classify_text

In [2]:
pd.set_option('max_colwidth', 800)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

## Preprocessing code

- Will use the below class and functions for both video transcripts.

---

1. `YouTubeTranscriptApi()` to fetch the YT transcripts.
2. `extract_data()` to transform data into Pandas DataFrame.
3. `clean_data()` to properly form sentences.

In [3]:
ytt_api = YouTubeTranscriptApi()

In [4]:
def extract_data(snippets) -> pd.DataFrame:
    """Extract the text, start time, and duration from the YT `FetchedTranscriptSnippet()`."""
    snippet_texts = []
    snippet_starts = []
    snippet_durations = []
    data = {}

    for snippets_idx in range(len(snippets)):
        snippet = snippets[snippets_idx]
        snippet_texts.append(snippet.text)
        snippet_starts.append(snippet.start)
        snippet_durations.append(snippet.duration)

    data['Text'] = snippet_texts
    data['Start Time'] = snippet_starts
    data['Duration'] = snippet_durations

    df = pd.DataFrame(data=data)
    return df

In [5]:
def clean_data(df) -> list[str]:
    """Rows can contain multiple sentences and some sentences can be on multiple rows, so ensure we 
    join proper sentences together.
    """

    text = df.Text.to_list()
    text_joined = ' '.join(text)
    # print(f"{text_joined}")
    text_split = text_joined.split('.')
    return text_split

## Pre-Game Clip

In [6]:
# PRE-GAME/PRE-FINAL 4
# video_id = "https://www.youtube.com/watch?v=-rjnvL9LL3U"
video_id = "-rjnvL9LL3U"
pre_game_snippets = ytt_api.fetch(video_id)
pre_game_snippets[:7]

[FetchedTranscriptSnippet(text='.', start=0.283, duration=1.248),
 FetchedTranscriptSnippet(text='THAT WILL BE A MONSTER GAME.', start=0.383, duration=5.879),
 FetchedTranscriptSnippet(text=">> LET'S HOPE EVERYBODY -- HELP", start=3.23, duration=8.195),
 FetchedTranscriptSnippet(text='EVERYBODY OUT MORE.', start=6.295, duration=9.744),
 FetchedTranscriptSnippet(text='THE 16 SEED HAS ONE TWO OF 24', start=11.458, duration=6.346),
 FetchedTranscriptSnippet(text='MEETINGS.', start=16.072, duration=4.864),
 FetchedTranscriptSnippet(text='ANALYTICS SAY ALL THE TOP THREE', start=17.837, duration=6.963)]

In [7]:
pre_game_snippets_df = extract_data(pre_game_snippets)
pre_game_snippets_df.head(3)

,Text,Start Time,Duration
0,.,0.283,1.248
1,THAT WILL BE A MONSTER GAME.,0.383,5.879
2,>> LET'S HOPE EVERYBODY -- HELP,3.230,8.195


In [8]:
pre_game_text = clean_data(pre_game_snippets_df)
pre_game_text[:7]

['',
 ' THAT WILL BE A MONSTER GAME',
 " >> LET'S HOPE EVERYBODY -- HELP EVERYBODY OUT MORE",
 ' THE 16 SEED HAS ONE TWO OF 24 MEETINGS',
 ' ANALYTICS SAY ALL THE TOP THREE SEEDS WILL ADVANCE',
 ' THE NINE SEED HAS WON TWO THIRDS OF THE GAMES AGAINST THE EIGHT SEED',
 ' YOU ALL HAD UCONN WINNING IT LAST YEAR']

## Post-Game Clip

In [9]:
# COMMENTATORS as game is ending
# https://www.youtube.com/watch?v=UHJ90jJmCYw
video_id = "UHJ90jJmCYw"
commentators_snippets = ytt_api.fetch(video_id)
commentators_snippets[:7]

[FetchedTranscriptSnippet(text='On the biggest stage, the ultimate goal,', start=1.199, duration=5.08),
 FetchedTranscriptSnippet(text='the national', start=4.24, duration=4.88),
 FetchedTranscriptSnippet(text='championship. Ron Groover, his fourth', start=6.279, duration=5.24),
 FetchedTranscriptSnippet(text="title game. Doug Surman's second", start=9.12, duration=5.399),
 FetchedTranscriptSnippet(text='championship game. Terry Oglesby fourth', start=11.519, duration=5.76),
 FetchedTranscriptSnippet(text='consecutive title game. Our officiating', start=14.519, duration=8.241),
 FetchedTranscriptSnippet(text='crew tonight in San Antonio. Florida and', start=17.279, duration=9.961)]

In [10]:
commentators_snippets_df = extract_data(commentators_snippets)
commentators_snippets_df = commentators_snippets_df[2028:] # shorten to end of game
commentators_snippets_df.head(3)

,Text,Start Time,Duration
2028,He can't touch it. He can't. One second,5294.480,5.199
2029,and it's over.,5297.520,5.119
2030,Florida is back on top of the college,5299.679,7.000


In [11]:
commentators_text = clean_data(commentators_snippets_df)
commentators_text[:7]

["He can't touch it",
 " He can't",
 " One second and it's over",
 ' Florida is back on top of the college basketball world championship madness',
 " It's a great day to be a Gator",
 ' They win the national title',
 ' Houston did not get a shot off']

## Prompt LLM

### Create Prompt

In [12]:
prompt_1 = """ 

Role: 
You are a linguist expert. You are acting as a prediction detector. Your task is to identify if a given sentence is a prediction about the future.

Background:
A prediction is a statement about what someone thinks will happen in the future.
Examples of predictions:
- "It will rain tomorrow." (Yes)
- "The stock market is expected to rise next quarter." (Yes)
- "I am going to the store." (No)
- “Lakers will win the championship. ”(Yes)

A prediction may contain: source, target, date, outcome.
"""

### Initialize LLMs

In [13]:
tgmf = TextGenerationModelFactory()

# Option 3: All NaviGator models
# models = tgmf.create_instances(tgmf.get_navigator_model_names())
models = tgmf.create_instances(['llama-3.1-8b-instant', 'llama-3.3-70b-versatile', 'granite-3.3-8b-instruct'])
models

### Pass data + prompt into each LLM

In [14]:
results = []
for text_split_idx in range(len(pre_game_text)):
    # print(text_split_idx, "/", len(pre_game_text))
    text = pre_game_text[text_split_idx]
    # if text_split_idx < 3:
    print(f"{text_split_idx} --- Sentence: {text}")
    for model in models:
        # print(model.__name__())
        raw_response, llm_label, llm_reasoning = llm_classify_text(text, prompt_1, model)
        result = (text, raw_response, llm_label, llm_reasoning, model.__name__())
        results.append(result)

        if text_split_idx < 3:
            print(f" Model: {model.__name__()} | Label:', {llm_label} | Reasoning:', {llm_reasoning}")

0 --- Sentence: 
 Model: llama-3.1-8b-instant | Label:', 0 | Reasoning:', Does not contain a future tense word or any information about the future
 Model: llama-3.3-70b-versatile | Label:', 0 | Reasoning:', There is no sentence provided to analyze
 Model: granite-3.3-8b-instruct | Label:', 1 | Reasoning:', Contains the future tense word 'will' and describes an event (temperature dropping) expected to happen tonight
1 --- Sentence:  THAT WILL BE A MONSTER GAME
 Model: llama-3.1-8b-instant | Label:', 1 | Reasoning:', Contains the future tense word 'will'
 Model: llama-3.3-70b-versatile | Label:', 1 | Reasoning:', Contains the future tense word 'will' indicating a prediction about the outcome of a future game
 Model: granite-3.3-8b-instruct | Label:', 1 | Reasoning:', The phrase 'will be' implies future tense, indicating a prediction about an upcoming event described as 'monster game'.
2 --- Sentence:  >> LET'S HOPE EVERYBODY -- HELP EVERYBODY OUT MORE
 Model: llama-3.1-8b-instant | Label

> - DataFrame below will have the same sentence (in text column) for all three llms (in the llm_name), then next set of rows will be another sentences with same llms.
> - llm_label column: {0: non-prediction, 1: prediction}

In [19]:
results_with_llm_label_df = pd.DataFrame(results, columns=['text', 'raw_response', 'llm_label', 'llm_reasoning', 'llm_name'])
results_with_llm_label_df['llm_label'] = pd.to_numeric(results_with_llm_label_df['llm_label'], errors='coerce').fillna(-1).astype(int)
results_with_llm_label_df.tail(6)

,text,raw_response,llm_label,llm_reasoning,llm_name
87,>> HE LEARNED NOT TO GO WITH MY PICK,"{""label"": 0, ""reasoning"": ""The sentence 'HE LEARNED NOT TO GO WITH MY PICK' implies a past action, not a prediction about the future.""}",0,"The sentence 'HE LEARNED NOT TO GO WITH MY PICK' implies a past action, not a prediction about the future.",llama-3.1-8b-instant
88,>> HE LEARNED NOT TO GO WITH MY PICK,"{""label"": 0, ""reasoning"": ""The sentence 'HE LEARNED NOT TO GO WITH MY PICK' is in the past tense, as indicated by the word 'LEARNED', and does not express a future outcome or event, thus it is not a prediction.""}",0,"The sentence 'HE LEARNED NOT TO GO WITH MY PICK' is in the past tense, as indicated by the word 'LEARNED', and does not express a future outcome or event, thus it is not a prediction.",llama-3.3-70b-versatile
89,>> HE LEARNED NOT TO GO WITH MY PICK,"{""label"": 0, ""reasoning"": ""The sentence '>> HE LEARNED NOT TO GO WITH MY PICK' is not a prediction because it describes a past action ('he learned') and provides advice or a lesson learned, rather than stating something that will happen in the future.""}",0,"The sentence '>> HE LEARNED NOT TO GO WITH MY PICK' is not a prediction because it describes a past action ('he learned') and provides advice or a lesson learned, rather than stating something that will happen in the future.",granite-3.3-8b-instruct
90,CLARK: YOU GUY,"{""label"": 0, ""reasoning"": ""Does not contain any future tense words or statements about expected outcomes.""}",0,Does not contain any future tense words or statements about expected outcomes.,llama-3.1-8b-instant
91,CLARK: YOU GUY,"{""label"": 0, ""reasoning"": ""The sentence 'CLARK: YOU GUY' does not contain any future tense words or phrases, and lacks a clear subject, target, date, or outcome, which are typical components of a prediction.""}",0,"The sentence 'CLARK: YOU GUY' does not contain any future tense words or phrases, and lacks a clear subject, target, date, or outcome, which are typical components of a prediction.",llama-3.3-70b-versatile
92,CLARK: YOU GUY,"{""label"": 0, ""reasoning"": ""The sentence 'CLARK: YOU GUY' does not contain any future tense words or phrases, nor does it express an expectation or forecast about future events. It is a statement addressing someone, hence it is not a prediction.""}",0,"The sentence 'CLARK: YOU GUY' does not contain any future tense words or phrases, nor does it express an expectation or forecast about future events. It is a statement addressing someone, hence it is not a prediction.",granite-3.3-8b-instruct


> - DataFrame above will have the same sentence (in text column) for all three llms (in the llm_name), then next set of rows will be another sentences with same llms.
> - llm_label column: {0: non-prediction, 1: prediction}

In [21]:
### Update DataFrame with model names (from llm_name column) as column names and the values (from llm_label column) be in model name column entry

def get_llm_labels(df, model_name):
    filt_llama = (df['llm_name'] == model_name)
    filt_df = df[filt_llama]
    return filt_df['llm_label']

results_df = pd.DataFrame(pre_game_text, columns=['Base Sentence'])
for model in models:
    print(model.__name__())
    model_labels = get_llm_labels(results_with_llm_label_df, model.__name__())
    results_df[model.__name__()] = model_labels.to_numpy().ravel()

results_df.head(3)

llama-3.1-8b-instant
llama-3.3-70b-versatile
granite-3.3-8b-instruct


,Base Sentence,llama-3.1-8b-instant,llama-3.3-70b-versatile,granite-3.3-8b-instruct
0,,0,0,1
1,THAT WILL BE A MONSTER GAME,1,1,1
2,>> LET'S HOPE EVERYBODY -- HELP EVERYBODY OUT MORE,0,0,0


### Majority Vote (across LLMs)

> - llm_label column: {0: non-prediction, 1: prediction}

In [22]:
majority_vote_label = results_df.mode(axis=1)
results_df['Majority Vote Label'] = majority_vote_label
results_df

,Base Sentence,llama-3.1-8b-instant,llama-3.3-70b-versatile,granite-3.3-8b-instruct,Majority Vote Label
0,,0,0,1,0
1,THAT WILL BE A MONSTER GAME,1,1,1,1
2,>> LET'S HOPE EVERYBODY -- HELP EVERYBODY OUT MORE,0,0,0,0
3,THE 16 SEED HAS ONE TWO OF 24 MEETINGS,0,0,0,0
4,ANALYTICS SAY ALL THE TOP THREE SEEDS WILL ADVANCE,1,1,1,1
5,THE NINE SEED HAS WON TWO THIRDS OF THE GAMES AGAINST THE EIGHT SEED,0,0,0,0
6,YOU ALL HAD UCONN WINNING IT LAST YEAR,0,0,1,0
7,WHO MAKES IT TO THE FINAL FOUR? AND WHO CUTS DOWN THE NETS? CLARK: I GET TO CHOOSE FIRST? I WILL TAKE MICHIGAN STATE,1,1,1,1
8,FLORIDA IS THE MOST COMPLETE TEAM IN THE FIELD,0,0,0,0
9,CLEMSON IS MY DARK HORSE,0,1,0,0


## Match Pre-Game with Commentators

In [23]:
filt_predictions = (results_df['Majority Vote Label'] == 1)
predictions_df = results_df[filt_predictions]
commentators_snippets_df

,Text,Start Time,Duration
2028,He can't touch it. He can't. One second,5294.480,5.199
2029,and it's over.,5297.520,5.119
2030,Florida is back on top of the college,5299.679,7.000
2031,basketball world championship,5302.639,7.761
2032,madness. It's a great day to be a Gator.,5306.679,7.000
...,...,...,...
2230,Nation. It's the third in Florida,5815.159,5.761
2231,"program history. 2006,",5818.000,5.380
2232,"2007, 2025.",5820.920,3.560
2233,[Applause],5823.380,3.900


In [24]:
commentators_text_df = pd.DataFrame(commentators_text, columns=['Commentators Text'])
commentators_text_df

,Commentators Text
0,He can't touch it
1,He can't
2,One second and it's over
3,Florida is back on top of the college basketball world championship madness
4,It's a great day to be a Gator
...,...
132,Florida wins the national championship
133,Congratulations to Gator Nation
134,It's the third in Florida program history
135,"2006, 2007, 2025"


In [25]:
### Manuall adding sentences as predictions bc the majority vote label is 0.

text = 'I AM GOING WITH THE GATORS AS MY CHAMP'

# Manual entries as majority vote really is 0
new_row = {
    'Base Sentence': text,
    'llama-3.1-8b-instant': 1,  # or whatever label value
    'llama-3.3-70b-versatile': 1,
    'granite-3.3-8b-instruct': 1,
    'Majority Vote Label': 1
}

predictions_df = pd.concat([predictions_df, pd.DataFrame([new_row])], ignore_index=True)
predictions_df

,Base Sentence,llama-3.1-8b-instant,llama-3.3-70b-versatile,granite-3.3-8b-instruct,Majority Vote Label
0,THAT WILL BE A MONSTER GAME,1,1,1,1
1,ANALYTICS SAY ALL THE TOP THREE SEEDS WILL ADVANCE,1,1,1,1
2,WHO MAKES IT TO THE FINAL FOUR? AND WHO CUTS DOWN THE NETS? CLARK: I GET TO CHOOSE FIRST? I WILL TAKE MICHIGAN STATE,1,1,1,1
3,I HAD IOWA STATE HERE BUT BECAUSE OF THE INJURY TO GILBERT I WILL TAKE THE SPARTANS,1,1,1,1
4,"I WILL TAKE DUKE AND FLORIDA IN THE FINAL AND THE BLUE DEVILS, ASSUMING THAT COOPER FLAGG IS HEALTHY, THEY ARE CUTTING DOWN THE NETS",1,1,1,1
5,"IN MY CHAMPIONSHIP GAME, I HAVE HOUSTON AND AUBURN AND I HAVE THE AUBURN TIGERS FINISHING THE SEASON AS NATIONAL CHAMPS",1,1,1,1
6,ADAM: I THOUGHT HE WOULD GO BIG EAST,1,1,1,1
7,>> THREE DIFFERENT CHAMPS FOR US,1,1,1,1
8,I AM GOING WITH THE GATORS AS MY CHAMP,1,1,1,1


## Get similarity of each prediction to all observations

In [41]:
import spacy
import pandas as pd

nlp = spacy.load('en_core_web_lg')
spacy_cs_results = []

for idx, row in predictions_df.iterrows():
    prediction_text = row['Base Sentence']
    prediction_doc = nlp(prediction_text)
    # if idx < 1:
        # print(f"Prediction: {prediction_doc}")
    
    for idx, row in commentators_text_df.iterrows():
        commentator_text = row['Commentators Text']
        commentator_doc = nlp(commentator_text)
        spacy_cs_pair_score = prediction_doc.similarity(commentator_doc)
        if idx < 1:
            print(f"How similiar is the prediction: {prediction_doc} \n\t   to the observation:  {commentator_doc}? --- {spacy_cs_pair_score}")
        
        spacy_cs_results.append({
            'prediction_text': prediction_text,
            'commentator_text': commentator_text,
            'similarity_score': spacy_cs_pair_score
        })

# Convert to DataFrame
spacy_cs_df = pd.DataFrame(spacy_cs_results)
spacy_cs_df

How similiar is the prediction:  THAT WILL BE A MONSTER GAME 
	   to the observation:  He can't touch it? --- 0.7869787812232971
How similiar is the prediction:  ANALYTICS SAY ALL THE TOP THREE SEEDS WILL ADVANCE 
	   to the observation:  He can't touch it? --- 0.7348655462265015
How similiar is the prediction:  WHO MAKES IT TO THE FINAL FOUR? AND WHO CUTS DOWN THE NETS? CLARK: I GET TO CHOOSE FIRST? I WILL TAKE MICHIGAN STATE 
	   to the observation:  He can't touch it? --- 0.8643709421157837
How similiar is the prediction:  I HAD IOWA STATE HERE BUT BECAUSE OF THE INJURY TO GILBERT I WILL TAKE THE SPARTANS 
	   to the observation:  He can't touch it? --- 0.8169417977333069
How similiar is the prediction:  I WILL TAKE DUKE AND FLORIDA IN THE FINAL AND THE BLUE DEVILS, ASSUMING THAT COOPER FLAGG IS HEALTHY, THEY ARE CUTTING DOWN THE NETS 
	   to the observation:  He can't touch it? --- 0.8042857050895691
How similiar is the prediction:  IN MY CHAMPIONSHIP GAME, I HAVE HOUSTON AND AUBUR

,prediction_text,commentator_text,similarity_score
0,THAT WILL BE A MONSTER GAME,He can't touch it,0.786979
1,THAT WILL BE A MONSTER GAME,He can't,0.742152
2,THAT WILL BE A MONSTER GAME,One second and it's over,0.803108
3,THAT WILL BE A MONSTER GAME,Florida is back on top of the college basketball world championship madness,0.783673
4,THAT WILL BE A MONSTER GAME,It's a great day to be a Gator,0.873409
...,...,...,...
1228,I AM GOING WITH THE GATORS AS MY CHAMP,Florida wins the national championship,0.620870
1229,I AM GOING WITH THE GATORS AS MY CHAMP,Congratulations to Gator Nation,0.682866
1230,I AM GOING WITH THE GATORS AS MY CHAMP,It's the third in Florida program history,0.734325
1231,I AM GOING WITH THE GATORS AS MY CHAMP,"2006, 2007, 2025",0.315503


> Sort by greatest to least similarity. Assuming this will match prediction to observation. Will have to pass both to LLM as well.

In [42]:
spacy_cs_df.sort_values(by="similarity_score", ascending=False)

,prediction_text,commentator_text,similarity_score
599,"I WILL TAKE DUKE AND FLORIDA IN THE FINAL AND THE BLUE DEVILS, ASSUMING THAT COOPER FLAGG IS HEALTHY, THEY ARE CUTTING DOWN THE NETS","Now, Walter, I know it wasn't the easiest game for you, but the feeling of being able to cut down the nets and hoist a national championship trophy, man",0.941252
325,WHO MAKES IT TO THE FINAL FOUR? AND WHO CUTS DOWN THE NETS? CLARK: I GET TO CHOOSE FIRST? I WILL TAKE MICHIGAN STATE,"Now, Walter, I know it wasn't the easiest game for you, but the feeling of being able to cut down the nets and hoist a national championship trophy, man",0.935545
569,"I WILL TAKE DUKE AND FLORIDA IN THE FINAL AND THE BLUE DEVILS, ASSUMING THAT COOPER FLAGG IS HEALTHY, THEY ARE CUTTING DOWN THE NETS","And Bird, how appropriate is it ends up a loose ball on the floor against Houston at Florida Got it",0.930039
319,WHO MAKES IT TO THE FINAL FOUR? AND WHO CUTS DOWN THE NETS? CLARK: I GET TO CHOOSE FIRST? I WILL TAKE MICHIGAN STATE,You guys lost in the first round and the turnaround you were able to do,0.928253
394,WHO MAKES IT TO THE FINAL FOUR? AND WHO CUTS DOWN THE NETS? CLARK: I GET TO CHOOSE FIRST? I WILL TAKE MICHIGAN STATE,We got multiple guys on this go that could go get a bucket and do anything,0.927547
...,...,...,...
102,THAT WILL BE A MONSTER GAME,Gator,0.190848
924,ADAM: I THOUGHT HE WOULD GO BIG EAST,Gator,0.181737
376,WHO MAKES IT TO THE FINAL FOUR? AND WHO CUTS DOWN THE NETS? CLARK: I GET TO CHOOSE FIRST? I WILL TAKE MICHIGAN STATE,Gator,0.157167
239,ANALYTICS SAY ALL THE TOP THREE SEEDS WILL ADVANCE,Gator,0.088571


In [43]:
filt_text = (spacy_cs_df["prediction_text"] == text) 
spacy_cs_df[filt_text].sort_values(by='similarity_score', ascending=False)

,prediction_text,commentator_text,similarity_score
1147,I AM GOING WITH THE GATORS AS MY CHAMP,"Now, Walter, I know it wasn't the easiest game for you, but the feeling of being able to cut down the nets and hoist a national championship trophy, man",0.873428
1217,I AM GOING WITH THE GATORS AS MY CHAMP,"So, my team held me down until I was able to put the ball in the basket",0.873014
1191,I AM GOING WITH THE GATORS AS MY CHAMP,"Hey, I know that I know that like a lot of coaches, you have dreamt of this moment",0.872034
1151,I AM GOING WITH THE GATORS AS MY CHAMP,"I can't wait to see my mama, man",0.871610
1175,I AM GOING WITH THE GATORS AS MY CHAMP,What does that mean to you? I I've just been incredibly blessed,0.860762
...,...,...,...
1103,I AM GOING WITH THE GATORS AS MY CHAMP,"Florida 65, Houston 63",0.422513
1154,I AM GOING WITH THE GATORS AS MY CHAMP,Congratulations,0.366400
1225,I AM GOING WITH THE GATORS AS MY CHAMP,Congratulations,0.366400
1198,I AM GOING WITH THE GATORS AS MY CHAMP,Gator,0.334264
